# 1. librerías necesarias para construir, entrenar y evaluar el modelo CNN MobileNetV2, además de herramientas para manejo de datos, visualización y métricas.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# 2. Definicion de Hiperparametros

In [ ]:
IMG_SIZE = (128, 512)
BATCH_SIZE = 64
EPOCHS = 20
LR_MOBILENET = 1e-4


# 3. Verificacion de rutas de datos

In [ ]:
base_dir = 'data/dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


# 4. Deteccion atuoamticas de clases 

In [ ]:
def get_classes_from_directory(directory):
    return sorted([
        d for d in os.listdir(directory)
        if os.path.isdir(os.path.join(directory, d)) and not d.startswith('.')
    ])

CLASSES = get_classes_from_directory(train_dir)
NUM_CLASSES = len(CLASSES)

print(CLASSES)

# 5. Aumentacion de datos para entrenamiento

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=False,  # Placas no deberían reflejarse
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255, dtype='float32')


## train_datagen = ImageDataGGenerator(..)

este generador solo usa el cojunto den entrenamiento

Argumentos utilizados: 

| Parámetro                     | Descripción                                                                                                   |
| ----------------------------- | ------------------------------------------------------------------------------------------------------------- |
| `rescale=1./255`              | Escala los píxeles de \[0, 255] a \[0, 1], necesario para redes neuronales.                                   |
| `rotation_range=5`           | Gira la imagen aleatoriamente entre -5° y 5°.                                                               |
| `width_shift_range=0.2`       | Desplaza la imagen horizontalmente hasta un 20% del ancho.                                                    |
| `height_shift_range=0.2`      | Desplaza la imagen verticalmente hasta un 20% de la altura.                                                   |
| `shear_range=0.15`            | Aplica una distorsión en forma de cizalla (shear), inclinando la imagen.                                      |
| `zoom_range=0.2`              | Aplica zoom aleatorio hasta un 20% más o menos.                                                               |
| `horizontal_flip=false`        | para la no asemitria.                                     |
| `fill_mode='nearest'`         | Cuando se hace una transformación que genera espacios vacíos, los rellena con el valor del píxel más cercano. |




basicamente lo que hace toma una imagen original y crea nuevas versiones modificadas aleatoriamente en cada epoca dele entrenamiento.

## val_test_datagen = ImageDataGenerator(..)

este lo utilizamos para validacion y test . no aplicamos ningun tipo de aumneto, ya que no queremos alterar los datos al evaluar el rendimiento del modelo 

1. rescale=1/255 -> normaliza los pixeles

2. dtype='float32' -> mismo tipo de dato del anterior

# 6. Generadores de datos

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=CLASSES
)

esta porcion de codigo leen las imagenes de disco , le asginana etiquetas segun la clase que es, la transforma y la entraga en lotes a la red neuronal para el entrenamiento la validacion y el testeo

# 7. Calculos de pesos por clases para balancer el entranamiento

In [ ]:
# Calcular class_weight
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights_array))
print("Pesos por clase:", class_weights)

Este código calcula pesos para cada clase según su frecuencia en los datos de entrenamiento, de forma que las clases menos frecuentes tengan un peso mayor. Esto ayuda a que el modelo no se sesgue hacia las clases más comunes.

1. compute_class_weight con 'balanced' genera esos pesos automáticamente.

2. class_weights es un diccionario {clase: peso} que luego se usa en el entrenamiento para darle más importancia a las clases minoritarias.

# 8. Construccion del modelo

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False

se carga el modelo preentrenado (sin la capa de clasificacion final) con pesos de imageNet. congelamosla capa convulacional para usarlo como extratctor de caracteristicas

In [ ]:
inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

1. creamos un tensor de entrada con la forma de la imagen 

2. pasamos esa entrada por el modelo base para extraer las caractetitsitcas 

3. aplicamos **GolbalAvergaePooling** para convertir el mapa de caracteristicas 2d un en vector 1D para poder pasarla a la capa siguiente

4. creamos la capa de salida con **NUM_CLASSES** neuronaes, con activacion softmax para clasificacion multiclase

In [ ]:
model = Model(inputs, outputs, name="MobileNetV2") # creamos el modelo entero con entrada y salida definida

Compilo el modelo con : 

1. optimizador Adam

2. la funcion de perdida es entropia cruzada 


3. metrica principal es el **accuracy**

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=LR_VGG),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# ![Descripción de la imagen](./img/mobile.png)

**OBS: SOLO LA CAPA DENSE ES ENTRENABLE**

In [ ]:
model.summary()

# 9.  Callbacks para control del entrenamiento

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
    ModelCheckpoint('output/best_mobilenet.h5', save_best_only=True, monitor='val_accuracy'),
]

1. EarlyStopping

    - Función: Detiene el entrenamiento si no mejora durante 3 épocas.

    - Ventaja: Evita el sobreentrenamiento y ahorra tiempo.

    - restore_best_weights=True: Recupera los mejores pesos al final.

2. ReduceLROnPlateau

    - Función: Reduce el learning rate si no mejora en 2 épocas.

    - Ventaja: Permite ajustes finos al estancarse el modelo.

    - factor=0.5: Reduce el LR al 50% del actual.

3. ModelCheckpoint

    - Función: Guarda el modelo cada vez que mejora la precisión en validación (val_accuracy).

    - Ventaja: Conserva automáticamente la mejor versión del modelo.

    - save_best_only=True: Solo guarda si mejora respecto al anterior.

# 10. Entranamiento del modelo

In [ ]:
# Entrenamiento
print("\nEntrenando MobileNetV2...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

# 11 . Visualizacion, Prediccion y evaluacion

In [ ]:
# Gráficos
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión - MobileNetV2')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida - MobileNetV2')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend()
plt.tight_layout()
plt.savefig('output/MobileNetV2_history.png')
plt.close()

# Evaluación
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
labels = list(test_generator.class_indices.values())
target_names = list(test_generator.class_indices.keys())
print("\nClassification Report for MobileNetV2:")
print(classification_report(
    y_true, y_pred_classes,
    labels=labels,
    target_names=target_names,
    digits=4,
    zero_division=0
))
cm = confusion_matrix(y_true, y_pred_classes, labels=labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.title('Matriz de Confusión - MobileNetV2')
plt.ylabel('Verdaderos')
plt.xlabel('Predicciones')
plt.savefig('output/MobileNetV2_confusion_matrix.png')
plt.close()

print("\nEntrenamiento y evaluación MobileNetV2 completados!")
